# SQLite

SQLite is an embedded, file-based relational database management system (RDBMS) that does not require any installation or setup. This makes it convenient to use since it does not need to run under a separate server process that needs to be started, stopped, or configured. In addition, a SQLite database is stored within a single file on your computer.


# `sqlite3` Library

The sqlite3 module provides an interface that adheres to the Python database API specification.
To access and manage our SQLite database, we can use the `sqlite3` module that is already included in the Python standard library. The library provides an SQL interface compliant with the DB-API 2.0 specification which you can read about in more detail in [PEP 249](https://www.python.org/dev/peps/pep-0249/). 

`sqlite3` [documentation](https://docs.python.org/3/library/sqlite3.html)

In [1]:
import sqlite3
import pandas as pd

# Databases

To begin, we need to pass the filename (and the path to the file if it is not located in the same folder) our database to the `sqlite3` library's `connect()` method, which will return a connection object that we can use to interact with our database.

If the database does not already exist, the `connect()` method will create an empty database for us.

`conn = sqlite3.connect('/path/to/file/database.sqlite3')`  
`conn = sqlite3.connect('/path/to/file/database.db')`

In [2]:
# Mac or Linux: Look at the available databases
# !ls -la Data

In [3]:
# Windows: Look at the available databases
# !dir Data

In [4]:
# Connect to a database
conn = sqlite3.connect('Data/company-database.db')

# Print the connection object
conn

In [5]:
# Mac or Linux: Verify the database was created
# !ls -la Data

In [6]:
# Windows: Verify the database was created
# !dir Data

## Cursor

Once we've connected to our database, we need to call the connection object's `cursor()` method. It will return a cursor object that includes several important methods we will need to interact with our database.

The first is the `cursor.execute()` method, which is what we will use to send our SQL statements to the SQLite database. Another is `fetchall()`. This method returns the results of our queries as a list with each row stored in a tuple.

Let's start by first creating our cursor object. 

In [7]:
# Create a cursor object
cursor = conn.cursor()

# Print the cursor object
cursor

# SQL Syntax

Some important points to keep in mind:
- Spaces, or whitespace, is treated differently than with Python, so you can use new lines and whitespace to format your code for readability.
- If the name of the table or column must consist of more than one word, use an underscore to connect them.
- SQL keywords are not case sensitive, but it is good practice to capitalize them so that you can easily identify them.
- You can use single or double quotes
- While SQLite does not, most relational database systems require that you end a query with a semicolon (`;`), so I recommend getting in the habit of doing this also.
- Avoid giving the same name to both a table and a column.

# Tables

Remember that a relational database stores data in tables. We can do that with the `CREATE TABLE` statement, which is where we will define the table name, column names, and the data types for each column.
 

## Data Types

When creating a new table, we must specify the data type of each column. Relational databases such as MySQL and MariaDB offer a much wider variety of data types. However, SQLite includes many of the core types you will need most often.

SQLite offers 5 different storage classes. A storage class is more general than a datatype but for the most part they are indistinguishable and the two terms are often used interchangeably.

| Data Type | Description |
| :-------- | :---------- |
| INTEGER | A whole number |
| REAL | An 8-byte floating-point value |
| BLOB | A blob of data (binary large object) for storing binary data. Ex. images |
| TEXT | A text string, typically UTF-8 encoded (depending on the database encoding) |
| NULL |  A NULL value representing missing data or an empty cell |

Unfortunately, SQLite does not include a `Boolean` data type. However, a simple workaround that is often used is to simply re-purpose the INTEGER type to represent those values (`0` = `false`, `1` = `true`).

It also does not have a datetime type. Instead, it leverages some built-in date and time functions so that we can use some of the other data types (`TEXT`, `REAL`, `INTEGER`) for storing date and time values. We'll look at some specific examples of how to do this a little later on.

*For more information on using dates and timestamps with SQLite, this tutorial has an excellent walkthrough [SQLite Date & Time](https://www.sqlitetutorial.net/sqlite-date/).*

**`CREATE TABLE` Syntax**

```
CREATE TABLE table_name (
    column_name    TEXT,
    column_name    INTEGER
);
```

In [8]:
# Create a table
cursor.execute("""CREATE TABLE employee (
        first_nme     TEXT,
        last_name     TEXT,
        title         TEXT,
        salary        INTEGER
);""")

## Duplicate Tables

Drats! One of our column names contains a typo. If we try to simply create a new table with the same name, we will get an `OperationalError`. One option is to delete the original table by using the `DROP TABLE` command and then create the new table.

In [9]:
# Drop an existing table
cursor.execute('DROP TABLE employee;')

# Create a table
cursor.execute("""CREATE TABLE employee (
        first_name    TEXT,
        last_name     TEXT,
        title         TEXT,
        salary        INT
);""")

However, what happens if we try to drop a table that doesn't exist?

In [10]:
# Try to drop a table that doesn't exist
cursor.execute('DROP TABLE employees;')

OperationalError: no such table: employees

To prevent receiving an error, adding `IF EXISTS` will allow our code to still run if we try to drop a table that does not exist. 

In [11]:
# Drop an existing table
cursor.execute('DROP TABLE IF EXISTS employees;')

## Table Names

The table names in a SQLite database are stored inside the system `sqlite_master` table. To access it, we will need first need to execute our query and then use the `fetchall()` method.

Don't worry that you haven't learned all of the SQL syntax in this code block yet! We will cover this syntax a little later in the lesson.

In [12]:
# Execute our query and fetch the results
cursor.execute("""
        SELECT name
        FROM sqlite_master
        WHERE type='table';
""").fetchall()

[('employee',)]

## INSERT INTO

`INSERT` statements are used to add a new row to a table in our database.

 *Please note that running the following cell multiple times will create duplicate rows.* 

In [13]:
# Drop an existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table
cursor.execute("""CREATE TABLE employee (
        first_name    TEXT,
        last_name     TEXT,
        title         TEXT,
        salary        INTEGER
);""")


# Insert table rows
cursor.execute("INSERT INTO employee VALUES ('Rupert', 'Giles', 'CEO', 425000);")
cursor.execute("INSERT INTO employee VALUES ('Willow', 'Rosenberg', 'Data Scientist', 350000);")

# Metadata

Metadata is information about the data in the database, such as the name of the tables and columns. We can access it in SQLite by using the `PRAGMA` command. 

For example, the `PRAGMA table_info(table_name)` command will return one row for each column in the table. Each column in the result set include the column's order number, name, data type, whether or not the column can be NULL, and its default value.

In [14]:
# Print out the table info
cursor.execute("PRAGMA TABLE_INFO('employee')").fetchall()

[(0, 'first_name', 'TEXT', 0, None, 0),
 (1, 'last_name', 'TEXT', 0, None, 0),
 (2, 'title', 'TEXT', 0, None, 0),
 (3, 'salary', 'INTEGER', 0, None, 0)]

In [15]:
# Query the column names
cursor.execute("PRAGMA TABLE_INFO('employee')")

# Save column names in a list 
names = [col[1] for col in cursor.fetchall()]
print(names)

['first_name', 'last_name', 'title', 'salary']


## Primary Key

Wait, how will our database know which column is our primary key?

When you create a new table and don't specify which column is the primary key, SQLite will automatically create an implicit auto-incremented column called `rowid`. It will store a signed integer that will uniquely identify each row in our table.

In [16]:
# SELECT all columns
cursor.execute("""
        SELECT *
        FROM employee;
""").fetchall()

[('Rupert', 'Giles', 'CEO', 425000),
 ('Willow', 'Rosenberg', 'Data Scientist', 350000)]

In [17]:
# Explicitly select the `rowid` field
cursor.execute("""
        SELECT rowid, first_name, last_name
        FROM employee;
""").fetchall()

[(1, 'Rupert', 'Giles'), (2, 'Willow', 'Rosenberg')]

Let's see how we can set our primary key when we creating a new table.

In [18]:
# Drop an existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table
cursor.execute("""CREATE TABLE employee (
        id            INTEGER  PRIMARY KEY,
        first_name    TEXT,
        last_name     TEXT,
        title         TEXT,
        salary        INTEGER
);""")


# Insert new rows into the table
cursor.execute("INSERT INTO employee VALUES ('Rupert', 'Giles', 'CEO', 425000);")
cursor.execute("INSERT INTO employee VALUES ('Willow', 'Rosenberg', 'Data Scientist', 350000);")

OperationalError: table employee has 5 columns but 4 values were supplied

Wait, what when wrong? The problem is that we need to include `NULL` as a placeholder when inserting new records in our table.

Notice also how we can use a single `INSERT INTO` statement to add more than one row at a time to our table.

In [19]:
# Drop an existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table
cursor.execute("""CREATE TABLE employee (
        id            INTEGER  PRIMARY KEY,
        first_name    TEXT,
        last_name     TEXT,
        title         TEXT,
        salary        INTEGER
);""")


# Insert table rows
cursor.execute("""
        INSERT INTO employee
        VALUES
          (NULL, 'Rupert', 'Giles', 'CEO', 425000),
          (NULL, 'Willow', 'Rosenberg', 'Data Scientist', 350000);
""")

In [20]:
# Print the table values
cursor.execute('SELECT * FROM employee;').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000)]

Another possible option is to define the fields you want to insert in your query and intentionally skip the autoincremented primary key field.

In [21]:
# Drop an existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table
cursor.execute("""CREATE TABLE employee (
        id            INTEGER  PRIMARY KEY,
        first_name    TEXT,
        last_name     TEXT,
        title         TEXT,
        salary        INTEGER
);""")


# Insert table rows
cursor.execute("""
        INSERT INTO employee (first_name, last_name, title, salary)
        VALUES
          ('Rupert', 'Giles', 'CEO', 425000),
          ('Willow', 'Rosenberg', 'Data Scientist', 350000);
""")

In [22]:
# Print the table values
cursor.execute('SELECT * FROM employee;').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000)]

Another way to insert multiple rows into a table is by using the cursor object's `executemany()` method, which takes two parameters&mdash;a parameterized SQL statement and the data in a tuple.

In addition, in this example notice how we are able to chain together the `execute()` and `fetchall()` methods. I've also purposely created two duplicate or nearly duplicate rows so that we can see how we can deal with them later on.

In [23]:
# Store data in a list
data = [
    ('Anya', 'Jenkins', 'CFO', 380000),
    ('Xander', 'Harris', 'Building Manager', 120500),
    ('Buffy', 'Summers', 'Security Specialist', 240000),
    ('Buffy', 'Summers', 'Security Specialist', 240000),
    ('Cordelia', 'Chase', 'Receptionist', 72000),
    ('Cordelia', 'Chase', 'Receptionist', 70000),
    ('Jesse', 'McNally', 'Waste Management Trainee', 55000)
]

# Insert multiple rows into a table
cursor.executemany('INSERT INTO employee (first_name, last_name, title, salary) VALUES(?, ?, ?, ?)', data)

# Query and return all the data in the table
cursor.execute("""
        SELECT *
        FROM employee;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000),
 (3, 'Anya', 'Jenkins', 'CFO', 380000),
 (4, 'Xander', 'Harris', 'Building Manager', 120500),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000),
 (9, 'Jesse', 'McNally', 'Waste Management Trainee', 55000)]

## ALTER TABLE

We can use `ALTER TABLE` to add new columns to our table.

In [24]:
# Add a new column without a default value
cursor.execute("""
        ALTER TABLE employee
        ADD COLUMN manager INTEGER;
""")

# Query and return all the data in the table
cursor.execute('SELECT * FROM employee').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, None),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, None),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, None),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, None),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, None),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, None),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, None),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, None),
 (9, 'Jesse', 'McNally', 'Waste Management Trainee', 55000, None)]

Since we didn't set a default value when adding our new column, we can see that SQLite automatically populated the value with `None`. Unfortunately, SQLite only recently added support for dropping a column, so this functionality is not yet available with the `sqlite3` library. However, this helps to demonstrate why it is helpful to store and structure your schema in a reusable script.

Let's recreate our table and also look at how use `INSERT INTO` to add multiple rows of data to our table in a single command.

In [25]:
# Drop the existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table schema
cursor.execute("""CREATE TABLE employee (
        id            INTEGER  PRIMARY KEY,
        first_name    TEXT     NOT NULL,
        last_name     TEXT     NOT NULL,
        title         TEXT     NOT NULL,
        salary        INTEGER  NOT NULL
);""")


# Insert the table rows
cursor.execute("INSERT INTO employee VALUES (NULL, 'Rupert', 'Giles', 'CEO', 425000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Willow', 'Rosenberg', 'Data Scientist', 350000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Anya', 'Jenkins', 'CFO', 380000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Xander', 'Harris', 'Building Manager', 120500);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Buffy', 'Summers', 'Security Specialist', 240000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Buffy', 'Summers', 'Security Specialist', 240000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Cordelia', 'Chase', 'Receptionist', 72000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Cordelia', 'Chase', 'Receptionist', 70000);")
cursor.execute("INSERT INTO employee VALUES (NULL, 'Jesse', 'McNally', 'Waste Management Trainee', 55000);")

In [26]:
# Drop the existing table
cursor.execute('DROP TABLE IF EXISTS employee;')

# Create a table schema
cursor.execute("""CREATE TABLE employee (
        id            INTEGER  PRIMARY KEY,
        first_name    TEXT     NOT NULL,
        last_name     TEXT     NOT NULL,
        title         TEXT     NOT NULL,
        salary        INTEGER  NOT NULL
);""")


# Insert table rows
cursor.execute("""
        INSERT INTO employee (id, first_name, last_name, title, salary)
        VALUES
          (NULL, 'Rupert', 'Giles', 'CEO', 425000),
          (NULL, 'Willow', 'Rosenberg', 'Data Scientist', 350000),
          (NULL, 'Anya', 'Jenkins', 'CFO', 380000),
          (NULL, 'Xander', 'Harris', 'Building Manager', 120500),
          (NULL, 'Buffy', 'Summers', 'Security Specialist', 240000),
          (NULL, 'Buffy', 'Summers', 'Security Specialist', 240000),
          (NULL, 'Cordelia', 'Chase', 'Receptionist', 72000),
          (NULL, 'Cordelia', 'Chase', 'Receptionist', 70000),
          (NULL, 'Jesse', 'McNally', 'Waste Management Trainee', 55000)
;""")

In [27]:
# Query and return all the data in the table
cursor.execute('SELECT * FROM employee').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000),
 (3, 'Anya', 'Jenkins', 'CFO', 380000),
 (4, 'Xander', 'Harris', 'Building Manager', 120500),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000),
 (9, 'Jesse', 'McNally', 'Waste Management Trainee', 55000)]

Now let's add the new column but set the default value to `0`. By doing this, the default value will be inserted for each existing row in our database and each time we add a new row to the table and do not include a value for it.

In [28]:
# Add a new column with a default row value
cursor.execute('ALTER TABLE employee ADD COLUMN manager TEXT DEFAULT 0;')

# Query and return all the data in the table
cursor.execute('SELECT * FROM employee').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '0'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '0'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '0'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '0'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '0'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0'),
 (9, 'Jesse', 'McNally', 'Waste Management Trainee', 55000, '0')]

## UPDATE 

Here is how we can use the `UPDATE` clause to change values for specific rows in our table. (We'll soon look at the `WHERE` clause in much more detail.)

In [29]:
# Update the manager values
cursor.execute("UPDATE employee SET manager = 1 WHERE last_name = 'Giles';")
cursor.execute("UPDATE employee SET manager = 1 WHERE last_name = 'Rosenberg';")
cursor.execute("UPDATE employee SET manager = 1 WHERE last_name = 'Jenkins';")
cursor.execute("UPDATE employee SET manager = 1 WHERE last_name = 'Summers';")

# Verify the values were changed
cursor.execute('SELECT * FROM employee;').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0'),
 (9, 'Jesse', 'McNally', 'Waste Management Trainee', 55000, '0')]

## DELETE

Unfortunately, Jesse McNally, our new Waste Management trainee, didn't last a single day! Let's see how we can remove his row from our `employee` table.

In [30]:
# Execute the SQL statement
cursor.execute("DELETE FROM employee WHERE first_name = 'Jesse'")

# Verify that the row was removed
cursor.execute('SELECT * FROM employee').fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

# SQL Queries

Most often, data analysts will not need to create and upload data to a new database. You'll likely find that the majority of your work will involve using SQL to query your data. Queries are SQL statements we construct to retrieve data from our database.

# SELECT

SQL queries will always start with the `SELECT`, followed by the name of the columns you want to include in your result set. The result set is only a temporary table, so it's important to keep in mind that our queries aren't modifying or changing any of the underlying data in our database. 

# FROM

After `SELECT`, the next clause we need to specify is `FROM`, which tells SQL which table in our database to query the data from. 

In [31]:
# Query and return all table columns
cursor.execute("""
        SELECT id, first_name, last_name, title, salary, manager
        FROM employee;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

**SELECT all columns**

Rather than type out all of the column names in our table, we can pass in the special `*` character. `SELECT *` will select all the columns from all the referenced tables (or subqueries) in the `FROM` clause.

In [32]:
# Query and return all table columns
cursor.execute("""
        SELECT *
        FROM employee;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

**Subset Columns**

If we don't want to return all of the columns in our table, we can specify exactly which ones we'd like returned in our `SELECT` statement.

In [33]:
# Query and return a subset of the table columns
cursor.execute("""
        SELECT last_name, first_name, salary
        FROM employee;
""").fetchall()

[('Giles', 'Rupert', 425000),
 ('Rosenberg', 'Willow', 350000),
 ('Jenkins', 'Anya', 380000),
 ('Harris', 'Xander', 120500),
 ('Summers', 'Buffy', 240000),
 ('Summers', 'Buffy', 240000),
 ('Chase', 'Cordelia', 72000),
 ('Chase', 'Cordelia', 70000)]

# Pandas Library

While the `sqlite3` module is conveniently included as part of the Python standard library, we are also able to use the pandas library.

One option that can make it easier to work with your query results is to save it directly into a pandas DataFrame.

In [34]:
# Query all rows in the table
cursor.execute('SELECT * FROM employee')

# Store the query results in a pandas dataframe
df = pd.DataFrame(cursor.fetchall())

# Save the column names
df.columns = [i[0] for i in cursor.description]

# Print the first 5 rows
df.head()

,id,first_name,last_name,title,salary,manager
0,1,Rupert,Giles,CEO,425000,1
1,2,Willow,Rosenberg,Data Scientist,350000,1
2,3,Anya,Jenkins,CFO,380000,1
3,4,Xander,Harris,Building Manager,120500,0
4,5,Buffy,Summers,Security Specialist,240000,1


However, pandas also includes the `read_sql()` method to store our query results directly in a DataFrame! First, we pass in the SQL query, and then the connection object we created earlier when we first connected to our database.

Pandas `pandas.read_sql` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)

In [35]:
# Save all the data from the `employee` table
df = pd.read_sql('SELECT * FROM employee;', con=conn)
# df = pd.read_sql('SELECT * FROM employee;', conn)

# Print the dataframe
df.head()

,id,first_name,last_name,title,salary,manager
0,1,Rupert,Giles,CEO,425000,1
1,2,Willow,Rosenberg,Data Scientist,350000,1
2,3,Anya,Jenkins,CFO,380000,1
3,4,Xander,Harris,Building Manager,120500,0
4,5,Buffy,Summers,Security Specialist,240000,1


Rather than passing in our SQL query directly to the `read_sql()` method, it can often be more convenient to first save it in a separate variable.

In [36]:
# Save the query as a string
query = '''SELECT *
            FROM employee;'''

# query = """SELECT *
#            FROM employee;"""

# Save the query as a pandas datafame
df = pd.read_sql(query, con=conn)

# Print the dataframe
df.head()

,id,first_name,last_name,title,salary,manager
0,1,Rupert,Giles,CEO,425000,1
1,2,Willow,Rosenberg,Data Scientist,350000,1
2,3,Anya,Jenkins,CFO,380000,1
3,4,Xander,Harris,Building Manager,120500,0
4,5,Buffy,Summers,Security Specialist,240000,1


# LIMIT Clause

If you are querying a large database, using `SELECT *` can be very computationally expensive and it's often not that helpful to print out a large number of rows. Instead, one option is to add a `LIMIT` clause to your query. For example, here we are tell SQL to return only the first five rows of our table. (*Please note: the LIMIT clause should also be added at the very end of your query*)

In [37]:
# Query and return all table columns
pd.read_sql("""
        SELECT * 
        FROM employee
        LIMIT 5;
""", conn)

,id,first_name,last_name,title,salary,manager
0,1,Rupert,Giles,CEO,425000,1
1,2,Willow,Rosenberg,Data Scientist,350000,1
2,3,Anya,Jenkins,CFO,380000,1
3,4,Xander,Harris,Building Manager,120500,0
4,5,Buffy,Summers,Security Specialist,240000,1


The `LIMIT` clause can accept one or two arguments. The `offset` parameter specifies the offset of the first row to return. The offset of the first row is 0, not 1. (The square brackets around `offset` are only intended to show that this parameter is optional.)

The `row_count` parameter specifies the maximum number of rows to return. It is mandatory, so when you use the `LIMIT` clause with only one argument, it will be used to determine the maximum number of rows to return from the first row of the result set.

The optional `offset` parameter specifies the offset of the first row to return, and if you do not pass in a value for this parameter it will be zero by default. Therefore, passing in `LIMIT 1` or `LIMIT 0, 1` will return the same results. (The values of both arguments must be zero or positive integers.)

```
SELECT column1, column2, …
FROM table_name
LIMIT [offset,] row_count;
```

In [38]:
# Query and return only the first row
cursor.execute("""
        SELECT *
        FROM employee
        LIMIT 0, 1;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1')]

In [39]:
# Query and return only the first row
cursor.execute("""
        SELECT *
        FROM employee
        LIMIT 1;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1')]

Here, we will offset the results by 1 row and then return the next 2 rows, so our query returns the second and third rows in our table.

In [40]:
# Return rows 2 and 3
cursor.execute("""
        SELECT *
        FROM employee
        LIMIT 1, 2;
""").fetchall()

[(2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1')]

# ORDER BY

We can use `ORDER BY` to sort the results of our queries. By default, it will sort the text alphabetically, like we see here. However, we can reverse the order by adding the optional `DESC` keyword.

In [41]:
# Select and order by the `last_name` column (ascending)
cursor.execute("""
        SELECT first_name, last_name
        FROM employee
        ORDER BY last_name;
""").fetchall()

[('Cordelia', 'Chase'),
 ('Cordelia', 'Chase'),
 ('Rupert', 'Giles'),
 ('Xander', 'Harris'),
 ('Anya', 'Jenkins'),
 ('Willow', 'Rosenberg'),
 ('Buffy', 'Summers'),
 ('Buffy', 'Summers')]

In [42]:
# Select and order by the `last_name` column (descending)
cursor.execute("""
        SELECT first_name, last_name
        FROM employee
        ORDER BY last_name DESC;
""").fetchall()

[('Buffy', 'Summers'),
 ('Buffy', 'Summers'),
 ('Willow', 'Rosenberg'),
 ('Anya', 'Jenkins'),
 ('Xander', 'Harris'),
 ('Rupert', 'Giles'),
 ('Cordelia', 'Chase'),
 ('Cordelia', 'Chase')]

When using `ORDER BY` on a numerical column it will sort the values from lowest to highest by default.

In [43]:
# Select and order by the `salary` column (ascending)
cursor.execute("""
        SELECT first_name, last_name, salary
        FROM employee
        ORDER BY salary;
""").fetchall()

[('Cordelia', 'Chase', 70000),
 ('Cordelia', 'Chase', 72000),
 ('Xander', 'Harris', 120500),
 ('Buffy', 'Summers', 240000),
 ('Buffy', 'Summers', 240000),
 ('Willow', 'Rosenberg', 350000),
 ('Anya', 'Jenkins', 380000),
 ('Rupert', 'Giles', 425000)]

In [44]:
# Select and order by the `salary` column (descending)
cursor.execute("""
        SELECT first_name, last_name, salary
        FROM employee
        ORDER BY salary DESC;
""").fetchall()

[('Rupert', 'Giles', 425000),
 ('Anya', 'Jenkins', 380000),
 ('Willow', 'Rosenberg', 350000),
 ('Buffy', 'Summers', 240000),
 ('Buffy', 'Summers', 240000),
 ('Xander', 'Harris', 120500),
 ('Cordelia', 'Chase', 72000),
 ('Cordelia', 'Chase', 70000)]

# SELECT DISTINCT

`SELECT DISTINCT` will only return the unique values in the specified column(s). For example, remember that our dataset set contains two duplicate or nearly duplicate rows. 

In [45]:
# Print the duplicate rows for reference
cursor.execute("""
        SELECT *
        FROM employee
        WHERE last_name = 'Summers' OR last_name = 'Chase';
""").fetchall()

[(5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

The following only removes rows that are exact duplicates.

In [46]:
# Remove any duplicate rows from our search results
cursor.execute("""
        SELECT DISTINCT first_name, last_name, title, salary
        FROM employee
        ORDER BY last_name;
""").fetchall()

[('Cordelia', 'Chase', 'Receptionist', 72000),
 ('Cordelia', 'Chase', 'Receptionist', 70000),
 ('Rupert', 'Giles', 'CEO', 425000),
 ('Xander', 'Harris', 'Building Manager', 120500),
 ('Anya', 'Jenkins', 'CFO', 380000),
 ('Willow', 'Rosenberg', 'Data Scientist', 350000),
 ('Buffy', 'Summers', 'Security Specialist', 240000)]

Since `SELECT DISTINCT` will only consider the columns that are passed to it, we can remove the duplicate rows for Buffy Summers and Cordelia Chase by only including the `first_name` and `last_name` columns.

In [47]:
# SELECT and ORDER BY the unique `first_name` and `last_name` values
cursor.execute("""
        SELECT DISTINCT first_name, last_name
        FROM employee
        ORDER BY last_name DESC;
""").fetchall()

[('Buffy', 'Summers'),
 ('Willow', 'Rosenberg'),
 ('Anya', 'Jenkins'),
 ('Xander', 'Harris'),
 ('Rupert', 'Giles'),
 ('Cordelia', 'Chase')]

# GROUP BY 

The optional `GROUP BY` clause allows us to group our rows by using the column values or expressions, returning one row for each group. For each group, we can apply an aggregate function, such as `SUM` or `COUNT`, that we will learn more about later. It will return only one row for each group.

The `GROUP BY` clause must follow the `FROM` (and any `WHERE` clauses). We can pass it a single column, like in this first example, or multiple columns or expressions, that we want to use as criteria to group our rows.

In this example, we've grouped the rows by the `last_name` field, so notice that it gives us the same results as when we use the second query using `SELECT DISTINCT`. 

In [48]:
# Group the rows by the `last_name` field
cursor.execute("""
        SELECT last_name
        FROM employee
        GROUP BY last_name
        ORDER BY last_name;
""").fetchall()

[('Chase',),
 ('Giles',),
 ('Harris',),
 ('Jenkins',),
 ('Rosenberg',),
 ('Summers',)]

In [49]:
# Return the distinct values in the `last_name` field
cursor.execute("""
        SELECT DISTINCT last_name
        FROM employee
        ORDER BY last_name;
""").fetchall()

[('Chase',),
 ('Giles',),
 ('Harris',),
 ('Jenkins',),
 ('Rosenberg',),
 ('Summers',)]

`GROUP BY` is often most helpful when used with aggregate functions, so you will see more helpful examples a little later on.

# WHERE Clause

Often you will find it is not very helpful to return all of the data from a table. Instead, what you're looking for is data that fits certain parameters, or meets certain conditions. We can do this with SQL by adding a `WHERE` clause to our query.

A `WHERE` clause is used to filter our results for rows that only meet our specific criteria. It will go through each row in our table and check if the condition evaluates to `true`, and if so, it returns the row in our result set. If it is `false`, the row is not included. 
- Always follows the `FROM` clause
- Search is case sensitive
- Enclose text in single or double quotes

```
SELECT column1, column2, …
FROM table_name
WHERE condition;
```

In [50]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE first_name = 'Buffy';
""").fetchall()

[(5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1')]

# Operators

In the previous search we used the equals (`=`) operator but there are several other operators available to help us filter our result set. For example, here we are using the "greater than" operator (`>`) to filter our results to only rows with a `salary` value greater than $200,000.

In [51]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE salary > 200000;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1')]

Another one we can use is the "not equal too" (`!=`) operator. 

In [52]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE salary != 240000;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

## AND

A `WHERE` clause may contain one or more conditions, separated by the operators `AND` and `OR`. 

If multiple conditions are separated by the `AND` operator, then all of the conditions must evaluate to `true` for the row to be included in the result set. 

In [53]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE salary = 240000 AND manager = 1;
""").fetchall()

[(5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1')]

Another operator we can use is `!=`. Here, we'll start by just filtering out rows where `salary` is equal to `240000`.

In [54]:
pd.read_sql("""
        SELECT first_name, last_name, salary, manager
        FROM employee
        WHERE salary != 240000;
""", conn)

,first_name,last_name,salary,manager
0,Rupert,Giles,425000,1
1,Willow,Rosenberg,350000,1
2,Anya,Jenkins,380000,1
3,Xander,Harris,120500,0
4,Cordelia,Chase,72000,0
5,Cordelia,Chase,70000,0


Next, we can filter our results even further by adding `AND` since both clauses must evaluate to `true` in order to be returned.

In [55]:
pd.read_sql("""
        SELECT first_name, last_name, salary, manager
        FROM employee
        WHERE salary != 240000 AND manager = 1;
""", conn)

,first_name,last_name,salary,manager
0,Rupert,Giles,425000,1
1,Willow,Rosenberg,350000,1
2,Anya,Jenkins,380000,1


## OR

If a `WHERE` clause contains an `OR` operator, then only one of the conditions must evaluate to `true` for the row to be included in the result set.

In [56]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE salary != 240000 OR manager = 1;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

## IN 

The `IN` operator is used in a `WHERE` clause to determine if the specified value matches any of the possible values. 

```
SELECT column1, column2,...
FROM table_name
WHERE (expr|column_1) IN ('value1', 'value2',...);
```

In [57]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title IN ('CEO', 'CFO');
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1')]

## NOT IN 

You can combine the `IN` and `NOT` operators to test whether the value does not match any of the possible options.

In [58]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title NOT IN ('CEO', 'CFO');
""").fetchall()

[(2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

## BETWEEN

`BETWEEN` is a logical operator that allows us to test whether a column value falls within a particular range.

In the following query, it returns `true` if the value of the expression is greater than or equal to (`>=`) the value of the starting value and less than or equal to (`<=`) the value of the second value.

In [59]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE salary BETWEEN 100000 AND 240000;
""").fetchall()

[(4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1')]

## NOT BETWEEN

The `NOT BETWEEN` operator returns rows for when the value is less than (`<`) the value of the starting expression or greater than (`>`) the value of ending expression.

In [60]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE salary NOT BETWEEN 240000 AND 350000;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

# Wildcards

Sometimes you may find that you only know part of the information that you need. In these case, it may be helpful to use wildcards to query our data. Wildcards are special symbols that represent unknown characters and they are used with the `LIKE` operator. The `LIKE` operator is used in the `WHERE` clauses to filter our results based on patterns.

The most commonly used wildcard is the `%`, which when placed in a query string matches any string of zero or more characters. It is not case sensitive. For example, `'Ca%a'` matches `Canada` and `california`.

In [61]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title LIKE 'c%';
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1')]

In [62]:
# Print the unique `title` values
cursor.execute("""
        SELECT DISTINCT title
        FROM employee
        ORDER BY title;
""").fetchall()

[('Building Manager',),
 ('CEO',),
 ('CFO',),
 ('Data Scientist',),
 ('Receptionist',),
 ('Security Specialist',)]

The underscore (`_`) wildcard is used to match any single character. 

In [63]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title LIKE 'cf_';
""").fetchall()

[(3, 'Anya', 'Jenkins', 'CFO', 380000, '1')]

**NOT LIKE**

In [64]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title NOT LIKE 'c%';
""").fetchall()

[(2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

# IS NULL Operator

`NULL` is special value since it indicates that a piece of information is unknown and two unknown values cannot be comparable or equal. This is why the following syntax is not correct:

```
SELECT *
FROM table
WHERE column = NULL;
```

Instead, to test whether a value is `NULL` or not, we can use the `IS NULL` operator. If the value is `NULL`, the expression returns `true` and returns the row. Otherwise, it returns `false` and is not included. Since our example table isn't missing any `'title'` values, there are no rows in our results table.

In [65]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title IS NULL;
""").fetchall()

[]

**IS NOT NULL Operator**

To only return rows where the value is not missing, or not `NULL`, we can use the `IS NOT NULL` operator.

In [66]:
cursor.execute("""
        SELECT *
        FROM employee
        WHERE title IS NOT NULL;
""").fetchall()

[(1, 'Rupert', 'Giles', 'CEO', 425000, '1'),
 (2, 'Willow', 'Rosenberg', 'Data Scientist', 350000, '1'),
 (3, 'Anya', 'Jenkins', 'CFO', 380000, '1'),
 (4, 'Xander', 'Harris', 'Building Manager', 120500, '0'),
 (5, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (6, 'Buffy', 'Summers', 'Security Specialist', 240000, '1'),
 (7, 'Cordelia', 'Chase', 'Receptionist', 72000, '0'),
 (8, 'Cordelia', 'Chase', 'Receptionist', 70000, '0')]

# Arithmetic Operators

So far, our queries have just been returning the raw data. However, SQLite also lets use several arithmetic operators: multiplication (`*`), division (`/`), addition (`+`), subtraction (`-`), and modulo (`%`).

In [67]:
# Integer divison
cursor.execute("""
        SELECT 9 / 2;
""").fetchall()

[(4,)]

In [68]:
# To return a floating point value, only one value must be floating point numbers
cursor.execute("""
        SELECT 9 / 2.0;
""").fetchall()

[(4.5,)]

In [69]:
# Division by zero is not allowed, so the expression returns NULL
cursor.execute("""
        SELECT 9 / 0;
""").fetchall()

[(None,)]

The modulo `%` operator finds the remainder of division of one number by another. For example, the following query returns 2 since 3 goes into 11 three times with a remainder of 2.

In [70]:
# The % operator is called the modulo operator. It finds the remainder of division of one number by another. 
cursor.execute("""
        SELECT 11 % 3;
""").fetchall()

[(2,)]

We can also apply these operators to every value in a particular column like you see here.

In [71]:
cursor.execute("""
        SELECT first_name, last_name, (salary * 2) 
        FROM employee;
""").fetchall()

[('Rupert', 'Giles', 850000),
 ('Willow', 'Rosenberg', 700000),
 ('Anya', 'Jenkins', 760000),
 ('Xander', 'Harris', 241000),
 ('Buffy', 'Summers', 480000),
 ('Buffy', 'Summers', 480000),
 ('Cordelia', 'Chase', 144000),
 ('Cordelia', 'Chase', 140000)]

# Functions

SQLite offers many different types of functions. I will cover some of the most commonly used ones here, but if you are interested in learning, please refer to this helpful guide: [SQLite Functions](https://www.sqlitetutorial.net/sqlite-functions/)


## COUNT

The `COUNT` function is especially helpful to quickly see how many rows are in your table.

In [72]:
cursor.execute("""
        SELECT COUNT(*) 
        FROM employee;
""").fetchall()

[(8,)]

We can also apply aggregate functions, such as `SUM`, `MIN`, `MAX`, and `AVG`, to our columns.

In [73]:
# Find the total employee salary
cursor.execute("""
        SELECT SUM(salary)
        FROM employee;
""").fetchall()

[(1897500,)]

In [74]:
# Find the minimum employee salary
cursor.execute("""
        SELECT MIN(salary)
        FROM employee;
""").fetchall()

[(70000,)]

# Aliases

In [75]:
# Use an alias to change the column name
df = pd.read_sql("""
        SELECT COUNT(*) AS 'Row Count'
        FROM employee;""", conn)

# Print the query results
df

,Row Count
0,8


# Commit Changes

If you have performed any operation on the database, other than sending queries, it is important to commit those changes once you are finished using the connection object's the `commit()` method.

## Total Changes

To first see how many changes you made to your database, you can use the connection object's `total_changes` attribute.

In [76]:
# Total number of database rows that were changed
conn.total_changes

37

You can commit your changes by using the connection object's `commit()` method.

In [77]:
# Commit any changes
conn.commit()

# Close the Connection

After committing your changes, you need to close the connection to the database.

In [78]:
# Close the database connection
conn.close()